In [ ]:
# Basic data manipulation
import pandas as pd
import numpy as np
import random

# Visualization
import matplotlib.pyplot as plt

# Sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.impute import SimpleImputer

# Classifiers
from catboost import CatBoostClassifier

# Additional encoders
import category_encoders as ce

# Stats
import scipy.stats as stats

# Dimensionality reduction
from sklearn.decomposition import (
    PCA,
    KernelPCA, 
    FastICA,
    TruncatedSVD
)

### First and foremost, merge two dbs into one

In [ ]:
train_transaction = pd.read_csv("data/train_transaction.csv")
train_identity = pd.read_csv("data/train_identity.csv")

# Merge both dataframes on 'TransactionID'
train = pd.merge(train_transaction, train_identity, on="TransactionID", how="left")

print(f"Rows in merged training set: {train.shape[0]}")
print(f"Columns in merged training set: {train.shape[1]}")
train.head()

### Perform an initial exploratory data analysis (EDA) by checking missing value percentages and examining the target distribution.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Identifying the target variable distribution and missing values for features

In [ ]:
# Missing value percentages per column
missing_percent = (train.isnull().sum() / len(train)) * 100
missing_percent = missing_percent.sort_values(ascending=False)
print("Missing percentages per column:")
print(missing_percent[missing_percent > 0])

# Distribution of the target variable 'isFraud'
train['isFraud'].value_counts().plot(kind='bar')
plt.title("Distribution of Fraudulent vs Non-Fraudulent Transactions")
plt.xlabel("isFraud")
plt.ylabel("Count")
plt.show()

In [ ]:
null_cols = [col for col in train.columns if train[col].isna().sum() > 0.9 * len(train)]
null_cols

In [ ]:
missing_df = train.copy(deep=True)
for col in null_cols:
    missing_df["m_flag_"+col] = np.where(missing_df[col].isnull(), 1, 0)
    correlation = missing_df[["m_flag_"+col, 'isFraud']].corr()
    print(correlation)


In [ ]:
categorical_features = train.select_dtypes(include=['object', 'category']).columns
for col in categorical_features:
    print(col, len(set(train[col])), set(train[col]))

### Find truly categorical values

Screen resolution values are true numerical values, while all other features are categorical in nature

In [ ]:
train[['Screen_Width', 'Screen_Height']] = train['id_33'].str.split('x', expand=True).astype(float)
train = train.drop(columns=['id_33'])

In [ ]:
cat_cols = train.select_dtypes(include=['object', 'category']).columns

# Identify candidate categorical features based on unique value counts
candidate_categorical = {}
# Set a threshold for maximum unique values
unique_threshold = 20

# Iterate over numeric columns to check unique value counts
for col in train.select_dtypes(include=['int64', 'float64']).columns:
    unique_vals = train[col].nunique()
    if (unique_vals < unique_threshold) and (col != "isFraud"):
        candidate_categorical[col] = unique_vals

# Print candidate categorical features
print("Candidate categorical features (numeric columns with few unique values):")
for col, count in candidate_categorical.items():
    print(f"{col}: {count} unique values")

### Imputing nulls

Not using standard imputation:
1. Placed zero values as indicator for missing values where feature values no zero values anywhere else
2. Added 'missing' instead of null for categorical values to keep all the columns

In [ ]:
# Identify numeric and categorical columns
num_cols = train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = list(set(cat_cols).union(set(candidate_categorical.keys())))
num_cols = [col for col in num_cols if col not in cat_cols and col not in ("TransactionID", "isFraud")]

# Imputation for numeric columns using zeros as indicator
num_imputer = SimpleImputer(strategy='constant', fill_value=0)
train[num_cols] = num_imputer.fit_transform(train[num_cols])

# Imputation for categorical columns using a constant value
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
train[cat_cols] = cat_imputer.fit_transform(train[cat_cols])

# Confirm that no missing values remain (or check overall missing count)
print("Total missing values after imputation:", train.isnull().sum().sum())

### Data encoding

In [ ]:
X = train.drop(columns=["isFraud", "TransactionID"])
y = train['isFraud']

### Encoding categorical features

Splitting train and test asap

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Using novel thing: WOE encoder to compensate for an enormous dimentionality for SVC

In [ ]:
encoder_high = ce.WOEEncoder(cols=cat_cols)
X_train_encoded_cat = encoder_high.fit_transform(X_train[cat_cols], y_train)
X_test_encoded_cat = encoder_high.transform(X_test[cat_cols])

### Data normalization

Scale data to remove any disrepancies in SVC

In [ ]:
scaler = StandardScaler()
X_train_scaled_num = scaler.fit_transform(X_train[num_cols])
X_test_scaled_num = scaler.transform(X_test[num_cols])

In [ ]:
X_train_scaled = pd.DataFrame(np.hstack([X_train_encoded_cat.values, X_train_scaled_num]), columns=cat_cols + num_cols)
X_test_scaled = pd.DataFrame(np.hstack([X_test_encoded_cat.values, X_test_scaled_num]), columns=cat_cols + num_cols)

In [ ]:
print("Encoded training set shape:", X_train_scaled.shape)
print("Encoded test set shape:", X_test_scaled.shape)

In [ ]:
X_train_scaled.head()

### Save data

In [ ]:
X_train_scaled.to_csv("data/preprocessed_train.csv", index=False)
X_test_scaled.to_csv("data/preprocessed_test.csv", index=False)

y_train.to_csv("data/y_train.csv", index=False)
y_test.to_csv("data/y_test.csv", index=False)

### Baseline catboost

Download processed data

In [ ]:
X_train_scaled = pd.read_csv("data/preprocessed_train.csv")
X_test_scaled = pd.read_csv("data/preprocessed_test.csv")
y_train = pd.read_csv("data/y_train.csv")
y_test = pd.read_csv("data/y_test.csv")

Accounting for severe class imbalance

In [ ]:
# Convert y_train to numpy array (since it's read as DataFrame)
y_train_array = y_train['isFraud'].values

# Get unique classes and compute weights
unique_classes = np.unique(y_train_array)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=unique_classes,
    y=y_train_array
)

# Create dictionary of class weights
class_weights_dict = dict(zip(unique_classes, class_weights))
class_weights_dict = {int(k): float(v) for k, v in class_weights_dict.items()}

PCA

In [ ]:
pca = PCA(n_components=100)
X_train_scaled = pca.fit_transform(X_train_scaled)
X_test_scaled = pca.transform(X_test_scaled)

Independent Component Analysis (ICA)

In [ ]:
ica = FastICA(n_components=100, random_state=42)
X_train_scaled = ica.fit_transform(X_train_scaled)
X_test_scaled = ica.transform(X_test_scaled)

Truncated SVD

In [ ]:
svd = TruncatedSVD(n_components=100, random_state=42)
X_train_scaled = svd.fit_transform(X_train_scaled)
X_test_scaled = svd.transform(X_test_scaled)

Train baseline model

In [ ]:
y_train_flat = y_train.values.ravel()
y_test_flat = y_test.values.ravel()

model_cat = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    class_weights=class_weights_dict,
    random_seed=42,
    verbose=100
)
model_cat.fit(X_train_scaled, y_train_flat)
y_pred_cat = model_cat.predict(X_test_scaled)
print("CatBoost Training Accuracy:", accuracy_score(y_test_flat, y_pred_cat))

#### With all features:
0:	learn: 0.6515508	total: 127ms	remaining: 1m 3s 

100:	learn: 0.3743155	total: 13.5s	remaining: 53.4s

200:	learn: 0.3282363	total: 26.9s	remaining: 40s

300:	learn: 0.2981647	total: 40s	remaining: 26.5s

400:	learn: 0.2745897	total: 53.3s	remaining: 13.2s

499:	learn: 0.2570046	total: 1m 6s	remaining: 0us

CatBoost Training Accuracy: **0.9161614793240085**

#### With PCA:
0:	learn: 0.6585685	total: 31.1ms	remaining: 15.5s

100:	learn: 0.4369775	total: 2.81s	remaining: 11.1s

200:	learn: 0.3974325	total: 5.63s	remaining: 8.38s

300:	learn: 0.3670191	total: 8.58s	remaining: 5.67s

400:	learn: 0.3430025	total: 11.6s	remaining: 2.86s

499:	learn: 0.3227418	total: 14.7s	remaining: 0us

CatBoost Training Accuracy: **0.8818962305686321**

#### With ISA:
0:	learn: 0.6616616	total: 32.2ms	remaining: 16.1s

100:	learn: 0.4328070	total: 3.35s	remaining: 13.2s

200:	learn: 0.3877602	total: 6.7s	remaining: 9.97s

300:	learn: 0.3543754	total: 10.1s	remaining: 6.69s

400:	learn: 0.3284518	total: 13.5s	remaining: 3.33s

499:	learn: 0.3067547	total: 16.8s	remaining: 0us

CatBoost Training Accuracy: **0.8889490974362448**

#### With SVD:
0:	learn: 0.6515508	total: 133ms	remaining: 1m 6s

100:	learn: 0.3743155	total: 13.2s	remaining: 52s

200:	learn: 0.3282363	total: 26.8s	remaining: 39.9s

300:	learn: 0.2981647	total: 40.3s	remaining: 26.6s

400:	learn: 0.2745897	total: 54.2s	remaining: 13.4s

499:	learn: 0.2570046	total: 1m 7s	remaining: 0us

CatBoost Training Accuracy: **0.9161614793240085**

### GA for input features subset selection

In [ ]:
X_train_scaled = pd.read_csv("data/preprocessed_train.csv")
X_test_scaled = pd.read_csv("data/preprocessed_test.csv")
y_train = pd.read_csv("data/y_train.csv")
y_test = pd.read_csv("data/y_test.csv")

In [ ]:
# Convert y_train to numpy array (since it's read as DataFrame)
y_train_array = y_train['isFraud'].values

# Get unique classes and compute weights
unique_classes = np.unique(y_train_array)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=unique_classes,
    y=y_train_array
)

# Create dictionary of class weights
class_weights_dict = dict(zip(unique_classes, class_weights))
class_weights_dict = {int(k): float(v) for k, v in class_weights_dict.items()}

In [ ]:
def run_genetic_algorithm(X_data, y_data, population_size=30, n_generations=20, subset_size=100):
    n_features = X_data.shape[1]
    
    # Initialize population - each individual is a sorted list of feature indices
    population = []
    for _ in range(population_size):
        subset = random.sample(range(n_features), subset_size)
        subset.sort()
        population.append(subset)
    
    def fitness(individual):
        X_subset = X_data.iloc[:, individual]
        
        # Manual train/test split instead of cross-validation
        X_train_subset, X_val_subset, y_train_subset, y_val_subset = train_test_split(
            X_subset, y_data, test_size=0.2, random_state=42, stratify=y_data
        )
        
        try:
            model = CatBoostClassifier(
                iterations=100,  # Reduce from 500 to speed up GA
                learning_rate=0.1,
                depth=6,
                class_weights=class_weights_dict,
                random_seed=42,
                verbose=0        # Turn off verbosity completely            
            )
            
            # Use a simple fit instead of cross_val_score
            model.fit(X_train_subset, y_train_subset, 
                     eval_set=(X_val_subset, y_val_subset),
                     early_stopping_rounds=20,
                     verbose=False)
            
            # Get validation accuracy
            accuracy = accuracy_score(y_val_subset, model.predict(X_val_subset))
            print(f"Feature subset evaluated: accuracy = {accuracy:.4f}")
            return accuracy
            
        except Exception as e:
            print(f"Error in fitness evaluation: {e}")
            return 0.0  # Return worst fitness on error
    
    # Creates a child by merging features from both parents and selecting a random subset
    def crossover(p1, p2, subset_size):
        combined = list(set(p1) | set(p2))  # Union of features
        if len(combined) > subset_size:
            child = sorted(random.sample(combined, subset_size))  # Ensure correct size
        else:
            child = sorted(combined)  # Keep all if below subset_size
        return child

    # Mutation replaces a random index in child if random.threshold is met
    def mutation(individual, n_features, subset_size):
        if random.random() < 0.1:  # 10% chance of mutation
            i = random.randrange(subset_size)
            available_features = set(range(n_features)) - set(individual)  # Exclude existing features
            if available_features:  
                new_feature = random.choice(list(available_features))
                individual[i] = new_feature
                individual.sort()
        return individual


    for i in range(n_generations):
        print(f"\nGeneration {i+1}/{n_generations}")
        # Evaluate fitness of population
        print("Evaluating fitness for each individual:")
        scored_population = []
        for idx, ind in enumerate(population):
            fitness_score = fitness(ind)
            scored_population.append((fitness_score, ind))
            print(f"Individual {idx+1}/{len(population)}: Fitness = {fitness_score:.4f}")
        
        scored_population.sort(key=lambda x: x[0], reverse=True)
        print(f"\nBest fitness in generation {i+1}: {scored_population[0][0]:.4f}")
        
        # Selection: truncation selection (pick top half as survivors)
        survivors = scored_population[: population_size // 2]
        
        # Then randomly select two parents (p1 & p2) from survivors for crossover + mutation
        print("Creating new population...")
        new_pop = [s[1] for s in survivors]
        while len(new_pop) < population_size:
            print("Generating new individual...")
            
            p1 = random.choice(survivors)[1]
            p2 = random.choice(survivors)[1]
            child = crossover(p1, p2, subset_size)
            child = mutation(child, n_features, subset_size)
            
            child = list(set(child))  # remove duplicates if any
            while len(child) < subset_size:  # if duplicates reduced size
                child.append(random.randrange(n_features))
            child.sort()
            new_pop.append(child)
            
            print("New individual created! Happy birthday!")
        population = new_pop
        print("Current best:", max([(fitness(ind), ind) for ind in population], key=lambda x: x[0])[1])
    
    best = max([(fitness(ind), ind) for ind in population], key=lambda x: x[0])[1]
    return best

best_features = run_genetic_algorithm(X_train_scaled, y_train)
X_train_ga = X_train_scaled.iloc[:, best_features]
X_test_ga = X_test_scaled.iloc[:, best_features]
model = CatBoostClassifier(
            iterations=500,
            learning_rate=0.1,
            depth=6,
            class_weights=class_weights_dict,
            random_seed=42,
            verbose=100
        )
model.fit(X_train_ga, y_train)
y_pred = model.predict(X_test_ga)
print("Model accuracy with selected features:", accuracy_score(y_test, y_pred))

### PSO and ACO for catboost hyperparameter tuning

In [ ]:
import numpy as np
from pyswarm import pso
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split # Optional: if creating validation set inside

def run_pso_for_catboost_hyperparams(X_train_ga, y_train, X_test_ga, y_test, n_particles=10, n_iterations=5):
    """
    Runs Particle Swarm Optimization (PSO) to find optimal hyperparameters for CatBoost using the provided training and testing data (already filtered by GA features).

    Args:
        X_train_ga: Training features (pandas DataFrame or numpy array) selected by GA.
        y_train: Training target variable.
        X_test_ga: Testing features (pandas DataFrame or numpy array) selected by GA.
        y_test: Testing target variable.
        n_particles: Number of particles in the swarm.
        n_iterations: Number of iterations for PSO.

    Returns:
        tuple: (best_params_dict, best_score)
               - best_params_dict: Dictionary containing the best hyperparameter set found.
               - best_score: The best AUC score achieved by PSO.
    """
    print(f"Starting PSO with {n_particles} particles and {n_iterations} iterations...")

    # Mapping for categorical parameters (indices):
    grow_policy_map = ['SymmetricTree', 'Depthwise', 'Lossguide']
    bootstrap_type_map = ['Bayesian', 'Bernoulli', 'MVS']
    leaf_estimation_method_map = ['Newton', 'Gradient']
    boosting_type_map = ['Ordered', 'Plain']

    param_bounds_pso = {
        # Key order must match the order in objective_function_pso's params array
        'learning_rate': (0.01, 0.3),             # params[0]
        'depth': (3, 10),                         # params[1] Integer
        'l2_leaf_reg': (1.0, 10.0),               # params[2]
        'iterations': (100, 1000),                # params[3] Integer
        'grow_policy_idx': (0, len(grow_policy_map) - 1), # params[4] Integer index
        'max_leaves': (4, 64),                    # params[5] Integer
        'min_data_in_leaf': (1, 50),              # params[6] Integer
        'random_strength': (0.1, 5.0),            # params[7]
        'bagging_temperature': (0.0, 1.0),        # params[8]
        'bootstrap_type_idx': (0, len(bootstrap_type_map) - 1), # params[9] Integer index
        'subsample': (0.5, 1.0),                  # params[10] Used only if bootstrap_type is not Bayesian
        'rsm': (0.5, 1.0),                        # params[11] aka colsample_bylevel
        'leaf_estimation_method_idx': (0, len(leaf_estimation_method_map) - 1), # params[12] Integer index
        'leaf_estimation_iterations': (1, 10),    # params[13] Integer
        'boosting_type_idx': (0, len(boosting_type_map) - 1), # params[14] Integer index
        'one_hot_max_size': (2, 30),              # params[15] Integer
        'early_stopping_rounds': (10, 50)         # params[16] Integer, used in fit
    }

    # Ordered list of keys corresponding to the parameter order
    param_keys = list(param_bounds_pso.keys())

    # Define the objective function for PSO
    def objective_function_pso(params):
        # --- Parameter Extraction and Mapping ---
        depth = int(round(params[param_keys.index('depth')]))
        iterations = int(round(params[param_keys.index('iterations')]))
        grow_policy_idx = int(round(params[param_keys.index('grow_policy_idx')]))
        max_leaves = int(round(params[param_keys.index('max_leaves')]))
        min_data_in_leaf = int(round(params[param_keys.index('min_data_in_leaf')]))
        bootstrap_type_idx = int(round(params[param_keys.index('bootstrap_type_idx')]))
        leaf_estimation_method_idx = int(round(params[param_keys.index('leaf_estimation_method_idx')]))
        leaf_estimation_iterations = int(round(params[param_keys.index('leaf_estimation_iterations')]))
        boosting_type_idx = int(round(params[param_keys.index('boosting_type_idx')]))
        one_hot_max_size = int(round(params[param_keys.index('one_hot_max_size')]))
        early_stopping_rounds = int(round(params[param_keys.index('early_stopping_rounds')]))

        grow_policy = grow_policy_map[grow_policy_idx]
        bootstrap_type = bootstrap_type_map[bootstrap_type_idx]
        leaf_estimation_method = leaf_estimation_method_map[leaf_estimation_method_idx]
        boosting_type = boosting_type_map[boosting_type_idx]

        # --- Build CatBoost Params ---
        cb_params = {
            'learning_rate': params[param_keys.index('learning_rate')],
            'depth': depth,
            'l2_leaf_reg': params[param_keys.index('l2_leaf_reg')],
            'iterations': iterations,
            'grow_policy': grow_policy,
            'max_leaves': max_leaves,
            'min_data_in_leaf': min_data_in_leaf,
            'random_strength': params[param_keys.index('random_strength')],
            'bagging_temperature': params[param_keys.index('bagging_temperature')],
            'bootstrap_type': bootstrap_type,
            'rsm': params[param_keys.index('rsm')], # colsample_bylevel
            'leaf_estimation_method': leaf_estimation_method,
            'leaf_estimation_iterations': leaf_estimation_iterations,
            'boosting_type': boosting_type,
            'one_hot_max_size': one_hot_max_size,
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'verbose': 0,
            'random_state': 42
        }

        if bootstrap_type in ['Bernoulli', 'MVS']:
            cb_params['subsample'] = params[param_keys.index('subsample')]
        # --- Optional: Create Validation Set ---
        # If you want to use early stopping on a validation set instead of the test set:
        # X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(
        #     X_train_ga, y_train, test_size=0.2, random_state=42, stratify=y_train
        # )
        # eval_set_data = (X_val_part, y_val_part)

        # --- Train and Evaluate ---
        model = CatBoostClassifier(**cb_params)

        try:
            # Using X_test_ga as eval set for early stopping
            model.fit(X_train_ga, y_train,
                      eval_set=(X_test_ga, y_test), # Or use eval_set_data if using validation split
                      early_stopping_rounds=early_stopping_rounds,
                      verbose=0)

            y_pred_proba = model.predict_proba(X_test_ga)[:, 1]
            auc = roc_auc_score(y_test, y_pred_proba)

        except Exception as e:
            print(f"Warning: Exception during model training/evaluation: {e}")
            print(f"Params causing issue: {cb_params}")
            # Return a very low score for problematic parameter combinations
            auc = 0.0

        # PSO maximizes, so return AUC directly
        return auc

    # Extract bounds for pyswarm
    lb = [param_bounds_pso[k][0] for k in param_keys]
    ub = [param_bounds_pso[k][1] for k in param_keys]

    # Run PSO
    best_params_pso_vals, best_score_pso = pso(objective_function_pso, lb, ub, swarmsize=n_particles, maxiter=n_iterations)

    # --- Process Results ---
    # Map best_params_pso_vals back to dictionary
    best_params_pso_dict = {}
    for i, key in enumerate(param_keys):
        val = best_params_pso_vals[i]
        if key in ['depth', 'iterations', 'grow_policy_idx', 'max_leaves', 'min_data_in_leaf',
                   'bootstrap_type_idx', 'leaf_estimation_method_idx', 'leaf_estimation_iterations',
                   'boosting_type_idx', 'one_hot_max_size', 'early_stopping_rounds']:
            best_params_pso_dict[key] = int(round(val))
        else:
            best_params_pso_dict[key] = val

    # Map indices back for categorical features
    best_params_pso_dict['grow_policy'] = grow_policy_map[best_params_pso_dict['grow_policy_idx']]
    best_params_pso_dict['bootstrap_type'] = bootstrap_type_map[best_params_pso_dict['bootstrap_type_idx']]
    best_params_pso_dict['leaf_estimation_method'] = leaf_estimation_method_map[best_params_pso_dict['leaf_estimation_method_idx']]
    best_params_pso_dict['boosting_type'] = boosting_type_map[best_params_pso_dict['boosting_type_idx']]

    # Store the early stopping rounds value separately as it's used in fit, not constructor
    best_early_stopping_rounds = best_params_pso_dict['early_stopping_rounds']

    # Remove index keys and early stopping rounds from the main dict
    keys_to_remove = ['grow_policy_idx', 'bootstrap_type_idx', 'leaf_estimation_method_idx', 'boosting_type_idx', 'early_stopping_rounds']
    for key in keys_to_remove:
       best_params_pso_dict.pop(key, None)

    # Handle conditional subsample: Remove if not needed, ensure exists if needed
    if best_params_pso_dict['bootstrap_type'] not in ['Bernoulli', 'MVS']:
        best_params_pso_dict.pop('subsample', None)
    elif 'subsample' not in best_params_pso_dict:
         # If needed but missing (e.g., boundary issue), assign a default or average
         best_params_pso_dict['subsample'] = np.mean(param_bounds_pso['subsample'])

    print("-" * 30)
    print("PSO Finished.")
    print(f"Best AUC score found by PSO: {best_score_pso:.6f}")
    print("Best CatBoost Parameters (excluding early_stopping_rounds):")
    for key, val in best_params_pso_dict.items():
        print(f"  {key}: {val}")
    print(f"Best early_stopping_rounds: {best_early_stopping_rounds}")
    print("-" * 30)

    # Add early stopping rounds back for potential direct use later if needed
    best_params_pso_dict_final = best_params_pso_dict.copy()
    best_params_pso_dict_final['early_stopping_rounds'] = best_early_stopping_rounds

    return best_params_pso_dict_final, best_score_pso

In [ ]:
import numpy as np
import random
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split # Optional

class ACO_HyperparameterOptimizer:
    """
    Ant Colony Optimization for Hyperparameter Tuning.

    Attributes:
        param_grid (dict): Dictionary where keys are hyperparameter names
                           and values are lists of discrete values to explore.
        objective_function (callable): A function that takes a dictionary of
                                      hyperparameters and returns a score to maximize.
        n_ants (int): Number of ants (solutions generated per iteration).
        n_iterations (int): Number of optimization iterations.
        alpha (float): Pheromone influence factor.
        beta (float): Heuristic influence factor (currently unused, set to 1).
        rho (float): Pheromone evaporation rate (0 < rho <= 1).
        Q (float): Pheromone deposit constant.
        min_pheromone (float): Minimum pheromone level to prevent stagnation.
        pheromones (dict): Nested dictionary storing pheromone levels for each parameter choice.
        global_best_score (float): Best score found across all iterations.
        global_best_params (dict): Hyperparameter set corresponding to the best score.
    """
    def __init__(self, param_grid, objective_function, n_ants, n_iterations,
                 alpha=1.0, beta=1.0, rho=0.1, Q=1.0, min_pheromone=0.01):
        """
        Initializes the ACO optimizer.
        """
        self.param_grid = param_grid
        self.objective_function = objective_function
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta  # Heuristic factor (often kept simple for HP tuning)
        self.rho = rho    # Evaporation rate
        self.Q = Q        # Pheromone deposit constant
        self.min_pheromone = min_pheromone # Minimum pheromone level

        self.param_names = list(self.param_grid.keys())
        self.pheromones = self._initialize_pheromones()

        self.global_best_score = -np.inf
        self.global_best_params = None

    def _initialize_pheromones(self):
        """
        Initializes pheromone trails with a constant value for all parameter choices.
        """
        pheromones = {}
        initial_pheromone = 1.0 # Start with a neutral pheromone level
        for param, values in self.param_grid.items():
            pheromones[param] = {value: initial_pheromone for value in values}
        return pheromones

    def _select_next_node(self, param_name):
        """
        Selects a value for a given parameter based on pheromone levels.
        Uses a probabilistic choice mechanism influenced by pheromones.
        """
        pheromone_values = self.pheromones[param_name]
        choices = list(pheromone_values.keys())
        current_pheromones = np.array([pheromone_values[choice] for choice in choices])

        # Heuristic information (tau^alpha * eta^beta)
        # For hyperparameter tuning, heuristic info (eta) is often uniform (beta=0 or eta=1)
        # unless prior knowledge exists. We'll use beta=1 but eta=1 implicitly.
        selection_probs = current_pheromones ** self.alpha # * (heuristic ** self.beta)

        # Handle potential division by zero if all probs are zero (e.g., early stage)
        prob_sum = np.sum(selection_probs)
        if prob_sum == 0:
            # If sum is zero, choose uniformly
            selection_probs = np.ones(len(choices)) / len(choices)
        else:
            selection_probs = selection_probs / prob_sum

        # Choose a value based on calculated probabilities
        chosen_value = np.random.choice(choices, p=selection_probs)
        return chosen_value

    def _construct_solution(self):
        """
        Constructs a complete hyperparameter set (solution) for a single ant.
        """
        solution = {}
        for param_name in self.param_names:
            solution[param_name] = self._select_next_node(param_name)
        return solution

    def _update_pheromones(self, ant_solutions, ant_scores):
        """
        Updates pheromone levels based on evaporation and deposition from ant solutions.
        """
        # 1. Evaporation
        for param, values in self.pheromones.items():
            for value in values:
                self.pheromones[param][value] *= (1.0 - self.rho)
                # Enforce minimum pheromone level
                self.pheromones[param][value] = max(self.pheromones[param][value], self.min_pheromone)

        # 2. Deposition
        # Deposit pheromone based on the quality of solutions found
        # More sophisticated strategies exist (e.g., only best ant deposits, elitism)
        # Here, all ants deposit based on their score
        for i in range(self.n_ants):
            solution = ant_solutions[i]
            score = ant_scores[i]

            # Normalize score or use directly? Depends on score range.
            # Assuming score is something like AUC (0 to 1), direct use might be okay.
            # We use Q as a scaling factor.
            if score > -np.inf: # Only deposit if the evaluation was successful
                delta_pheromone = self.Q * score # Simple proportional deposit

                for param_name, value in solution.items():
                    # Check if the parameter/value exists (it should)
                    if param_name in self.pheromones and value in self.pheromones[param_name]:
                        self.pheromones[param_name][value] += delta_pheromone
                    else:
                        # This case should ideally not happen if grid is consistent
                         print(f"Warning: Parameter '{param_name}' or value '{value}' not found in pheromone trails during deposition.")


    def run(self):
        """
        Executes the ACO optimization process.
        """
        print(f"Starting ACO: {self.n_iterations} iterations, {self.n_ants} ants/iteration.")
        print(f"Params: alpha={self.alpha}, beta={self.beta}, rho={self.rho}, Q={self.Q}")

        for iteration in range(self.n_iterations):
            ant_solutions = []
            ant_scores = []

            # Generate solutions for all ants
            for ant in range(self.n_ants):
                solution = self._construct_solution()
                try:
                    score = self.objective_function(solution.copy()) # Pass a copy
                    if score is None or not np.isfinite(score):
                       # Handle cases where objective function fails or returns invalid score
                       print(f"Warning: Objective function returned invalid score ({score}) for params: {solution}. Assigning low score.")
                       score = -np.inf
                except Exception as e:
                    print(f"Error evaluating solution: {solution}")
                    print(f"Exception: {e}")
                    score = -np.inf # Penalize errors heavily

                ant_solutions.append(solution)
                ant_scores.append(score)

                # Update global best if current ant is better
                if score > self.global_best_score:
                    self.global_best_score = score
                    self.global_best_params = solution
                    print(f"Iteration {iteration+1}, Ant {ant+1}: New best score! -> {self.global_best_score:.6f}")

            # Update pheromone trails based on the iteration's results
            self._update_pheromones(ant_solutions, ant_scores)

            avg_score = np.mean([s for s in ant_scores if s > -np.inf]) if any(s > -np.inf for s in ant_scores) else -np.inf
            print(f"Iteration {iteration+1}/{self.n_iterations} finished. Avg Score: {avg_score:.6f}. Best Score so far: {self.global_best_score:.6f}")

        print("-" * 30)
        print("ACO Finished.")
        if self.global_best_params:
            print(f"Best score found: {self.global_best_score:.6f}")
            print("Best parameters found:")
            for param, value in self.global_best_params.items():
                print(f"  {param}: {value}")
        else:
            print("ACO did not find a valid solution.")
        print("-" * 30)

        return self.global_best_params, self.global_best_score

In [ ]:
# Import the necessary libraries (CatBoostClassifier, roc_auc_score, etc.)
# Make sure the ACO_HyperparameterOptimizer class definition is above this function

def run_aco_for_catboost_hyperparams(X_train_ga, y_train, X_test_ga, y_test,
                                     n_ants=10, n_iterations=5,
                                     alpha=1.0, beta=1.0, rho=0.1, Q=1.0): # Add ACO params
    """
    Runs Ant Colony Optimization (ACO) using the implemented class to find
    optimal hyperparameters for CatBoost using the provided GA-filtered data.

    Args:
        X_train_ga: Training features (pandas DataFrame or numpy array) selected by GA.
        y_train: Training target variable.
        X_test_ga: Testing features (pandas DataFrame or numpy array) selected by GA.
        y_test: Testing target variable.
        n_ants (int): Number of ants (solutions per iteration).
        n_iterations (int): Number of iterations for ACO.
        alpha (float): Pheromone influence factor for ACO.
        beta (float): Heuristic influence factor for ACO (currently unused).
        rho (float): Pheromone evaporation rate for ACO.
        Q (float): Pheromone deposit constant for ACO.


    Returns:
        tuple: (best_params_dict, best_score)
               - best_params_dict: Dictionary containing the best hyperparameter set found.
               - best_score: The best AUC score achieved by ACO.
    """

    # --- Define Parameter Grid (same as before) ---
    param_grid_aco = {
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3],
        'depth': [3, 4, 5, 6, 7, 8, 9, 10],
        'l2_leaf_reg': [1.0, 3.0, 5.0, 7.0, 10.0],
        'iterations': [100, 200, 300, 500, 700, 1000],
        'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
        'max_leaves': [8, 16, 32, 64],
        'min_data_in_leaf': [1, 5, 10, 20, 50],
        'random_strength': [0.1, 0.5, 1.0, 2.0, 5.0],
        'bagging_temperature': [0.0, 0.2, 0.5, 0.8, 1.0],
        'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], # Used only if bootstrap_type is not Bayesian
        'rsm': [0.6, 0.7, 0.8, 0.9, 1.0], # colsample_bylevel
        'leaf_estimation_method': ['Newton', 'Gradient'],
        'leaf_estimation_iterations': [1, 3, 5, 10],
        'boosting_type': ['Ordered', 'Plain'],
        'one_hot_max_size': [2, 10, 20, 30],
        'early_stopping_rounds': [10, 20, 30, 50] # Used in fit
    }

    # --- Define Objective Function (same as before) ---
    def objective_function_aco(params):
        # params is a dictionary constructed by the ACO framework for one ant
        cb_params = params.copy()
        early_stopping_rounds = cb_params.pop('early_stopping_rounds', 30) # Default if missing

        bootstrap_type = cb_params.get('bootstrap_type', 'Bayesian')
        if bootstrap_type not in ['Bernoulli', 'MVS']:
            cb_params.pop('subsample', None)
        elif 'subsample' not in cb_params:
            # Handle missing subsample if required by bootstrap_type
            # Give it a default value from the grid if ACO didn't pick it
             median_subsample = np.median(param_grid_aco['subsample'])
             # print(f"Warning: Subsample needed for {bootstrap_type} but not chosen by ant. Setting to median: {median_subsample}")
             cb_params['subsample'] = median_subsample


        cb_params.update({
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'verbose': 0,
            'random_state': 42
        })

        # --- Optional: Validation Split ---
        # X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(...)
        # eval_set_data = (X_val_part, y_val_part)

        model = CatBoostClassifier(**cb_params)

        try:
            model.fit(X_train_ga, y_train,
                      eval_set=(X_test_ga, y_test), # Or use eval_set_data
                      early_stopping_rounds=early_stopping_rounds,
                      verbose=0)
            y_pred_proba = model.predict_proba(X_test_ga)[:, 1]
            auc = roc_auc_score(y_test, y_pred_proba)
            # Ensure we don't return NaN/inf scores which break ACO updates
            if not np.isfinite(auc):
                auc = 0.0 # Or some other low penalty score
        except Exception as e:
            # print(f"Warning: Exception during model training/evaluation: {e}")
            # print(f"Params causing issue: {cb_params}")
            auc = 0.0 # Penalize errors

        return auc

    # --- Instantiate and Run ACO ---
    aco_optimizer = ACO_HyperparameterOptimizer(
        param_grid=param_grid_aco,
        objective_function=objective_function_aco,
        n_ants=n_ants,
        n_iterations=n_iterations,
        alpha=alpha,
        beta=beta,
        rho=rho,
        Q=Q
    )

    best_params_aco, best_score_aco = aco_optimizer.run() # Run the optimization

    # --- Process and Return Results ---
    # The run method already prints results. We just return them.
    # Note: best_params_aco will include 'early_stopping_rounds' if found.
    return best_params_aco, best_score_aco

### Run PSO and ACO optimizations

In [ ]:
# Run PSO optimization
print("Starting PSO optimization...")
best_pso_params, best_pso_score = run_pso_for_catboost_hyperparams(
    X_train_ga, y_train, X_test_ga, y_test,
    n_particles=10,
    n_iterations=5
)

# Run ACO optimization
print("\nStarting ACO optimization...")
best_aco_params, best_aco_score = run_aco_for_catboost_hyperparams(
    X_train_ga, y_train, X_test_ga, y_test,
    n_ants=10,
    n_iterations=5,
    alpha=1.0,
    beta=1.0,
    rho=0.1,
    Q=1.0
)

### Train and evaluate final models with optimized parameters

In [ ]:
# Train final model with PSO parameters
final_esr_pso = best_pso_params.pop('early_stopping_rounds')
final_model_pso = CatBoostClassifier(
    **best_pso_params,
    random_state=42,
    loss_function='Logloss',
    eval_metric='AUC'
)
final_model_pso.fit(
    X_train_ga, y_train,
    eval_set=(X_test_ga, y_test),
    early_stopping_rounds=final_esr_pso,
    verbose=100
)

# Train final model with ACO parameters
final_esr_aco = best_aco_params.pop('early_stopping_rounds')
final_model_aco = CatBoostClassifier(
    **best_aco_params,
    random_state=42,
    loss_function='Logloss',
    eval_metric='AUC'
)
final_model_aco.fit(
    X_train_ga, y_train,
    eval_set=(X_test_ga, y_test),
    early_stopping_rounds=final_esr_aco,
    verbose=100
)

### Compare PSO and ACO results

In [ ]:
%pip install seaborn

In [ ]:
from sklearn import metrics
import seaborn as sns

def evaluate_and_visualize(model, X_train, X_test, y_train, y_test, title_prefix=""):
    """
    Comprehensive model evaluation with visualizations
    
    Args:
        model: Trained model object
        X_train: Training features
        X_test: Test features
        y_train: Training labels
        y_test: Test labels
        title_prefix: String to prepend to plot titles
        
    Returns:
        dict: Dictionary containing various performance metrics
    """
    # Get predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = metrics.accuracy_score(y_test, y_pred)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)
    avg_precision = metrics.average_precision_score(y_test, y_pred_proba)
    
    # 1. Print Classification Report
    print("\nClassification Report:")
    print(metrics.classification_report(y_test, y_pred, digits=4))
    
    # 2. Plot Confusion Matrix
    plt.figure(figsize=(8, 6))
    cm = metrics.confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Fraud', 'Fraud'],
                yticklabels=['Not Fraud', 'Fraud'])
    plt.title(f'{title_prefix}Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    
    # 3. ROC Curve
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2,
             label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{title_prefix}Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()
    
    # 4. Precision-Recall Curve
    precision, recall, _ = metrics.precision_recall_curve(y_test, y_pred_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2,
             label=f'P-R curve (AP = {avg_precision:.4f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'{title_prefix}Precision-Recall Curve')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()
    
    # 5. Feature Importance Plot (top 20)
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.get_feature_importance()
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=feature_importance.head(20),
                x='importance', y='feature', palette='viridis')
    plt.title(f'{title_prefix}Top 20 Feature Importance')
    plt.xlabel('Importance Score')
    plt.tight_layout()
    plt.show()
    
    # 6. Learning Curves
    history = pd.DataFrame(model.get_evals_result()['validation'])
    plt.figure(figsize=(10, 5))
    for metric in history.columns:
        plt.plot(history.index, history[metric], label=metric)
    plt.title(f'{title_prefix}Learning Curves')
    plt.xlabel('Iteration')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Return metrics dictionary
    return {
        'accuracy': accuracy,
        'roc_auc': roc_auc,
        'avg_precision': avg_precision,
        'confusion_matrix': cm,
        'f1_score': metrics.f1_score(y_test, y_pred)
    }

# Evaluate PSO model
print("Evaluating PSO-tuned model:")
metrics_pso = evaluate_and_visualize(
    final_model_pso,
    X_train_ga, X_test_ga,
    y_train, y_test,
    title_prefix="PSO-Tuned CatBoost: "
)

# Evaluate ACO model
print("\nEvaluating ACO-tuned model:")
metrics_aco = evaluate_and_visualize(
    final_model_aco,
    X_train_ga, X_test_ga,
    y_train, y_test,
    title_prefix="ACO-Tuned CatBoost: "
)

# Compare results
results_comparison = pd.DataFrame({
    'PSO': metrics_pso,
    'ACO': metrics_aco
}).round(4)
print("\nPSO vs ACO Performance Comparison:")
print(results_comparison)

### Publication-Quality Visualizations for PSO and ACO Comparison

In [ ]:
import seaborn as sns
from scipy import stats

# Set style for publication-quality plots
plt.style.use('seaborn-paper')
sns.set_context("paper", font_scale=1.5)

# Custom color palette
colors = ['#2ecc71', '#e74c3c']  # Green for PSO, Red for ACO
sns.set_palette(colors)

# 1. ROC Curves Comparison
plt.figure(figsize=(10, 8))
for model, name, color in [(final_model_pso, 'PSO', colors[0]), 
                          (final_model_aco, 'ACO', colors[1])]:
    y_pred_proba = model.predict_proba(X_test_ga)[:, 1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2,
             label=f'{name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison: PSO vs ACO')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_comparison.pdf', dpi=300, bbox_inches='tight')
plt.show()

# 2. Precision-Recall Curves
plt.figure(figsize=(10, 8))
for model, name, color in [(final_model_pso, 'PSO', colors[0]), 
                          (final_model_aco, 'ACO', colors[1])]:
    y_pred_proba = model.predict_proba(X_test_ga)[:, 1]
    precision, recall, _ = metrics.precision_recall_curve(y_test, y_pred_proba)
    avg_precision = metrics.average_precision_score(y_test, y_pred_proba)
    plt.plot(recall, precision, color=color, lw=2,
             label=f'{name} (AP = {avg_precision:.3f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves: PSO vs ACO')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('pr_comparison.pdf', dpi=300, bbox_inches='tight')
plt.show()

# 3. Feature Importance Comparison
def plot_feature_importance_comparison(pso_model, aco_model, X_train, top_n=15):
    # Get feature importances
    pso_importances = pd.DataFrame({
        'feature': X_train.columns,
        'importance': pso_model.get_feature_importance(),
        'model': 'PSO'
    })
    
    aco_importances = pd.DataFrame({
        'feature': X_train.columns,
        'importance': aco_model.get_feature_importance(),
        'model': 'ACO'
    })
    
    # Combine and get top features
    all_importances = pd.concat([pso_importances, aco_importances])
    top_features = all_importances.groupby('feature')['importance'].mean().nlargest(top_n).index
    
    # Filter for top features
    plot_data = all_importances[all_importances['feature'].isin(top_features)]
    
    plt.figure(figsize=(12, 8))
    sns.barplot(data=plot_data, x='importance', y='feature', hue='model')
    plt.title(f'Top {top_n} Feature Importance Comparison')
    plt.xlabel('Importance Score')
    plt.ylabel('Feature')
    plt.legend(title='')
    plt.tight_layout()
    plt.savefig('feature_importance_comparison.pdf', dpi=300, bbox_inches='tight')
    plt.show()

plot_feature_importance_comparison(final_model_pso, final_model_aco, X_train_ga)

# 4. Learning Curves Comparison
plt.figure(figsize=(12, 6))
pso_history = pd.DataFrame(final_model_pso.get_evals_result()['validation'])
aco_history = pd.DataFrame(final_model_aco.get_evals_result()['validation'])

plt.plot(pso_history.index, pso_history['AUC'], label='PSO', color=colors[0])
plt.plot(aco_history.index, aco_history['AUC'], label='ACO', color=colors[1])
plt.xlabel('Iteration')
plt.ylabel('AUC Score')
plt.title('Learning Curves: PSO vs ACO')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('learning_curves_comparison.pdf', dpi=300, bbox_inches='tight')
plt.show()

# 5. Performance Metrics Table
metrics_comparison = pd.DataFrame({
    'Metric': ['AUC-ROC', 'Average Precision', 'Accuracy', 'F1-Score'],
    'PSO': [metrics_pso['roc_auc'], metrics_pso['avg_precision'], 
            metrics.accuracy_score(y_test, final_model_pso.predict(X_test_ga)),
            metrics.f1_score(y_test, final_model_pso.predict(X_test_ga))],
    'ACO': [metrics_aco['roc_auc'], metrics_aco['avg_precision'],
            metrics.accuracy_score(y_test, final_model_aco.predict(X_test_ga)),
            metrics.f1_score(y_test, final_model_aco.predict(X_test_ga))]
})

# Create a styled table
plt.figure(figsize=(10, 4))
plt.axis('off')
table = plt.table(cellText=metrics_comparison.values,
                 colLabels=metrics_comparison.columns,
                 cellLoc='center',
                 loc='center',
                 colColours=['#f8f9fa'] * 3)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 1.8)
plt.title('Performance Metrics Comparison', pad=20)
plt.tight_layout()
plt.savefig('metrics_table.pdf', dpi=300, bbox_inches='tight', 
            transparent=True)
plt.show()

# 6. Statistical Significance Testing
def bootstrap_comparison(pso_model, aco_model, X_test, y_test, n_iterations=1000):
    pso_scores = []
    aco_scores = []
    n_samples = len(y_test)
    
    for _ in range(n_iterations):
        # Bootstrap sampling
        indices = np.random.randint(0, n_samples, n_samples)
        X_bootstrap = X_test.iloc[indices]
        y_bootstrap = y_test.iloc[indices]
        
        # Get predictions and compute AUC
        pso_pred = pso_model.predict_proba(X_bootstrap)[:, 1]
        aco_pred = aco_model.predict_proba(X_bootstrap)[:, 1]
        
        pso_scores.append(metrics.roc_auc_score(y_bootstrap, pso_pred))
        aco_scores.append(metrics.roc_auc_score(y_bootstrap, aco_pred))
    
    # Statistical test
    t_stat, p_value = stats.ttest_rel(pso_scores, aco_scores)
    
    # Visualization
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=pd.DataFrame({'PSO': pso_scores, 'ACO': aco_scores}))
    plt.title('Bootstrap Distribution of AUC Scores\n' + 
              f'p-value: {p_value:.4f}')
    plt.xlabel('AUC Score')
    plt.ylabel('Density')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('bootstrap_comparison.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    
    return t_stat, p_value

t_stat, p_value = bootstrap_comparison(final_model_pso, final_model_aco, 
                                     X_test_ga, y_test)
print(f"\nStatistical Comparison Results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")